In [ ]:
import Evaluation as eva

### SVM

In [ ]:
eva.evaluate("SVM","train_joined")

In [ ]:
eva.evaluate("SVM_missing_value","train_joined_missing_value")

In [ ]:
eva.evaluate("SVM_redundant","train_joined_redundant_value")

In [ ]:
eva.evaluate("SVM_undersample","train_joined_undersampled_value")

### LR

In [ ]:
eva.evaluate("LR","train_joined")

### XGboost

In [ ]:
eva.evaluate("XGboost","train_joined")